# Remapping

UXarray allows us to remap from one dataset to another, using various methods covered in this guide. This lets us take data from one grid, and transfer it to a different grid, which is very useful when we want to change grid resolution but keep the data of our current grid. It can also be used to take several different datasets, and combined them together to have a dataset with multiple different variables of data. This are just some of the more common use cases, there are of course many others.

In [ ]:
import uxarray as ux

# Visualization requirements
import geocat.datafiles as geodf
import cartopy.crs as ccrs
import geoviews.feature as gf
import holoviews as hv
import warnings

plot_opts = {"width": 700, "height": 350}

hv.extension("bokeh")

warnings.filterwarnings("ignore")

We will use the following datasets:

In [ ]:
datafiles = (
    geodf.get(
        "netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/diag.2016-08-20_00.00.00_subset.nc"
    ),
    geodf.get("netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc"),
)
grid_path = "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug"
data_path = "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc"

source = ux.open_dataset(datafiles[1], datafiles[0])
destination = ux.open_dataset(grid_path, data_path)

The features are for visualazation and is not related to remapping

In [ ]:
features = gf.coastline(
    projection=ccrs.PlateCarree(), line_width=1, scale="50m"
) * gf.states(projection=ccrs.PlateCarree(), line_width=1, scale="50m")

### Nearest Neighbor

Nearest neighbor remapping a point by point remapping which doesn't modify the data points as it remaps. This method takes and finds the nearest neighbor in the source grid to each point in the destination grid, and assigns that data value to the point in the destination grid. Lets take a look at our current destination dataset:

In [ ]:
destination

We can see that it only has a single variable, `psi` inside it. Now let's remap using nearest neighbor and take a variable from our source dataset, `relhum_200hPa`, and add it to our destination dataset.

`nearest_neighbor` has a few parameters we can use to control the remapping:

* `destination_obj` is the grid/dataset object that is being remapped to.
* `remap_to` specifices the location of the remaping, either to `nodes`, `face centers`, or `edge centers`.
* `coord_type` refers to what coordinate system to use, either `spherical` or `cartesian`.

In [ ]:
source["relhum_200hPa"].remap.nearest_neighbor(destination, remap_to="face centers")

As we can see, `relhum_200hPa` is now inside the destination dataset alongside `psi`. This can be visualized below, with the orginal source dataset being plotted alongside the new remapped data.

In [ ]:
source["relhum_200hPa"][0].plot.rasterize(
    method="polygon",
    exclude_antimeridian=True,
    title="Original Data",
    height=350,
    width=700,
) * features

In [ ]:
destination["relhum_200hPa"][0].plot.rasterize(
    method="polygon",
    exclude_antimeridian=True,
    title="Remapped Data",
    height=350,
    width=700,
) * features

The remapped data is much lower resolution, as the grid we remapped to is lower resolution itself.

### Inverse Distance Weighted

Inverse distance weighted remapping assigns a value based on the weighted average of a specified number of nearby points. This gives a more smooth remapping than the nearest neighbor and helps decrease the chance of outliers. However, it is a non-conservative remapping method.

In [ ]:
source["relhum_200hPa"].remap.inverse_distance_weighted(
    destination, remap_to="face centers"
)

In [ ]:
destination["relhum_200hPa"][0].plot.rasterize(
    method="polygon",
    exclude_antimeridian=True,
    title="Remapped Data",
    height=350,
    width=700,
) * features

For inverse distance weighted remapping the parameters are the same as nearest neighbor but, with two extra parameters we can use to change the remapping parameters.

* `power` controls how local or global the remapping is, the larger this value the less influence points that are further away have.
* `k` is the number of neighbors to use in the weighted average.

### Bilinear Remapping

In [ ]:
# Coming Soon